# Ensemble Challenge: Timepoint 2

Goal: to capture the complexity and nuances around the evolution of the pandemic at various stages and locations.

Location A: New York State
Consider the following settings:
Timepoint 1: April 3, 2020. Setting: New York State at the beginning of the pandemic when masking was the main preventative measure. No vaccines available.

Timepoint 2: July 15, 2021. Setting: New York State upon the arrival of the Delta variant. Vaccines available.

Timepoint 3: January 4, 2022. Setting: New York State coinciding with the arrival of the first Omicron wave. At-home testing widely available.

BONUS: Consider the same three time points, but change the setting to Texas, which had different COVID-19 dynamics compared to the Northeastern states.

...and related questions for each:
What is the most relevant data to use for model calibration?
What was our understanding of COVID-19 viral mechanisms at the time? For example, early in the pandemic, we didn't know if reinfection was a common occurance, or even possible.
What are the parameters related to contagiousness/transmissibility and severity of the dominant strain at the time?
What policies were in place for a stated location, and how can this information be incorporated into models? (See https://www.bsg.ox.ac.uk/research/covid-19-government-response-tracker for time series of interventions).
Also--this is a good source for parameters: https://www.cdc.gov/coronavirus/2019-ncov/hcp/planning-scenarios.html

For each timepoint:
(a) Take a single model, calibrate it using any historical data prior to the given date, and create a 4-week forecast for cases, hospitalizations, and deaths beginning on the given date. (b) Evaluate the forecast using the COVID-19 Forecasting Hub Error Metrics (WIS, MAE). The single model evaluation should be done in the same way as the ensemble.

Repeat (1), but with an ensemble of different models.

a. It is fine to calibrate each model independently and weight naively.

b. It would also be fine to calibrate the ensemble as a whole, assigning weights to the different component models, so that you minimize the error of the ensemble vs. historical data.

c. Use the calibration scores and error metrics computed by the CDC Forecasting Hub. As stated on their website:

“Periodically, we evaluate the accuracy and precision of the ensemble forecast and component models over recent and historical forecasting periods. Models forecasting incident hospitalizations at a national and state level are evaluated using adjusted relative weighted interval scores (WIS, a measure of distributional accuracy), and adjusted relative mean absolute error (MAE), and calibration scores. Scores are evaluated across weeks, locations, and targets. You can read a paper explaining these procedures in more detail, and look at the most recent monthly evaluation reports. The final report that includes case and death forecast evaluations is 2023-03-13.”

Produce the forecast outputs in the format specified by the CDC forecasting challenge, including the specified quantiles.

Data
Use the following data sources:

Cases: Johns Hopkins, Reich Lab (pulled from Johns Hopkins, but formatted)

Hospitalizations: HealthData.gov

Deaths: Johns Hopkins, Reich Lab

### Load dependencies

In [1]:
import os
import pandas as pd
import numpy as np
from pyciemss.Ensemble.interfaces import (
    load_and_sample_petri_ensemble, load_and_calibrate_and_sample_ensemble_model
)
from pyciemss.PetriNetODE.interfaces import (
    load_and_sample_petri_model,
    load_and_calibrate_and_sample_petri_model,
    load_and_optimize_and_sample_petri_model,
    load_and_calibrate_and_optimize_and_sample_petri_model
)
from pyciemss.visuals import plots

## Get data

In [2]:
url = 'https://raw.githubusercontent.com/DARPA-ASKEM/experiments/main/thin-thread-examples/milestone_12month/evaluation/ensemble_eval_SA/datasets/aabb3684-a7ea-4f60-98f1-a8e673ad6df5/dataset.csv'
ny_data = pd.read_csv(url)

# Grab test data for four-week forecast (07/15/2021 - 05/01/2020)
test_data = ny_data[0:569].drop(columns="timestep")

# Select historical data up to Timepoint 2, 07/14/2021 (the first 541 rows)
ny_data = ny_data[0:540]
ny_data[["I", "H", "D"]].to_csv("NY_data2.csv")

## Set up timepoints

In [3]:
start_timepoint = 0
stop_timepoint = 540 + 28 # simulate for four weeks after end of data
timepoints = [float(i) for i in range(stop_timepoint + 1)]

## Select relevant models

In [4]:
model1_location = "../../notebook/ensemble_eval_sa/SEIRHDS_V1.json"
model2_location = "../../notebook/ensemble_eval_sa/SEIRHDS_basic_config.json"
model3_location = "../../notebook/ensemble_eval_sa/SIRHD_age_stratified.json"
model4_location = "../../notebook/ensemble_eval_sa/SIRHD_vacc_var_V1.json"
model5_location = "../../notebook/ensemble_eval_sa/SIRHD_age_vacc_var.json"
model6_location = "../../notebook/ensemble_eval_sa/SIRHD_mask_V3.json"
model7_location = "../../notebook/ensemble_eval_sa/SIRHD_V1C.json"
model8_location = "../../notebook/ensemble_eval_sa/SEIRD_ymo_age_strat.json"

## Load, calibrate and sample an ensemble of one model

In [5]:
num_samples = 2
model_paths = [model1_location]
data_path = "../../notebook/ensemble_eval_sa/NY_data2.csv"
weights = [1]
solution_mappings = [{"I": "I", "H": "H", "D": "D"}] # "column name in data": "observable or state variable in model"

# Run the calibration and sampling
result = load_and_calibrate_and_sample_ensemble_model(
    model_paths,
    data_path,
    weights,
    solution_mappings,
    num_samples,
    timepoints,
    verbose=True,
    total_population=19340000,
    num_iterations=100,
    time_unit="days",
    visual_options={"title": "Calibrated Ensemble", "subset":".*_sol"}
)

# # Save results
# result["data"].to_csv(
#     os.path.join(DEMO_PATH, "results_petri_ensemble/calibrated_sample_results.csv"), index=False
# )
# result["quantiles"].to_csv(
#     os.path.join(DEMO_PATH, "results_petri_ensemble/calibrated_quantile_results.csv"), index=False
# )

# Plot results
schema = plots.trajectories(pd.DataFrame(result["data"]), subset=".*_sol",
                            points=test_data.reset_index(drop=True).rename(columns={"I":"I_data", "H":"H_data", "D":"D_data"}),
                           )
schema = plots.pad(schema, 5)
plots.ipy_display(schema)

iteration 0: loss = 45751.34715896845
iteration 25: loss = 28361.542366564274
iteration 50: loss = 24309.46016174555
iteration 75: loss = 24229.12921744585



## Load, calibrate and sample an ensemble of several models

In [6]:
num_samples = 2
model_paths = [model1_location, model4_location, model6_location]
data_path = "../../notebook/ensemble_eval_sa/NY_data2.csv"
weights = [1/3, 1/3, 1/3]
solution_mappings = [{"I": "I", "H": "H", "D": "D"},
                     {"I": "I", "H": "H", "D": "D"}, 
                     {"I": "Cases", "H": "Hosp", "D": "Deaths"},
                     ]

# Run the calibration and sampling
result = load_and_calibrate_and_sample_ensemble_model(
    model_paths,
    data_path,
    weights,
    solution_mappings,
    num_samples,
    timepoints,
    verbose=True,
    total_population=19340000,
    num_iterations=200,
    time_unit="days",
    visual_options={"title": "Calibrated Ensemble", "subset":".*_sol"}
)

# # Save results
# result["data"].to_csv(
#     os.path.join(DEMO_PATH, "results_petri_ensemble/calibrated_sample_results.csv"), index=False
# )
# result["quantiles"].to_csv(
#     os.path.join(DEMO_PATH, "results_petri_ensemble/calibrated_quantile_results.csv"), index=False
# )

# Plot results
schema = plots.trajectories(pd.DataFrame(result["data"]), subset=".*_sol",
                            points=test_data.reset_index(drop=True).rename(columns={"I":"I_data", "H":"H_data", "D":"D_data"}),
                           )
schema = plots.pad(schema, 5)
plots.ipy_display(schema)

ERROR:root:
                ###############################

                There was an exception in pyciemss
                
                Error occured in function: load_and_calibrate_and_sample_ensemble_model

                Function docs : 
    Load a collection petri net from a file, compile them into an ensemble probabilistic program, calibrate it on data,
    and sample from the calibrated model.

    Args:
        petri_model_or_paths: Iterable[Union[str, mira.metamodel.TemplateModel, mira.modeling.Model]
            - Each element of the iterable is a path to a petri net file, or a petri net object.
            - This path can be a URL or a local path to a mira model or AMR model.
            - Alternatively, this can be a mira template model directly.
        data_path: str
            - The path to the data to calibrate the model to. See notebook/integration_demo/data.csv
              for an example of the format.
            - The data should be a csv with one column

ValueError: Model file not found: ../../notebook/ensemble_eval_sa/SIRHD_vacc_var_V1.json